In [1]:
import sys
import os
import pandas as pd


# Add project root to sys.path in Jupyter or interactive session
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../../")))

from config.path_config import DATA_DIR, PROJECT_DIR, DEPENDENCIES_DIR
from src.helper_functions.data_pre_processing.data_upload_processor.processor import *
from src.helper_functions.database_upload.postgres_uploader import *

from dotenv import load_dotenv

In [3]:
os.getcwd()

'c:\\Users\\73315\\OneDrive - Bain\\Documents\\CL_Chatbot_V_01\\src\\helper_functions\\tools'

In [7]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

In [10]:
from query_mapping.query_mapping_functions import *
from tools import *
from helper_functions.data_pre_processing.data_upload_processor.processor import run_question_data_pipeline

In [11]:
yaml_file_path = "testing_mapping.yaml"
print(yaml_file_path)

testing_mapping.yaml


In [12]:
raw_data = pd.read_excel(os.path.join(DATA_DIR, "wave_10_responses.xlsx"))
data_map = pd.read_excel(os.path.join(DATA_DIR, "wave_10_question_mapping.xlsx"))

In [13]:
env_path = os.path.join(DEPENDENCIES_DIR, ".env")
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_openai import OpenAI

In [ ]:
def embed_partition(partition_df, model, embedding_output_dimensions):
    client = OpenAI()
    partition_df = partition_df.copy()
    partition_df["embedding"] = partition_df["text"].apply(
        lambda text: client.embeddings.create(
            input=[text.replace("\n", " ")],
            model=model,
            dimensions=embedding_output_dimensions
        ).data[0].embedding
    )
    return partition_df
 
 
def get_embeddings_from_column_dask(column_vector, embedding_output_dimensions=1536, model="text-embedding-3-small"):
    """
    Convert a column vector of text data into embeddings using Dask for chunked parallelism.
 
    Parameters:
    column_vector (pd.Series): A pandas Series containing text data.
    embedding_output_dimensions (int): Dimensionality of embeddings.
    model (str): OpenAI model name.
 
    Returns:
    list: A list of embeddings.
    """
    # Convert to Dask dataframe
    ddf = dd.from_pandas(column_vector.to_frame(name="text"), npartitions=32)
 
    # Apply the embedding function using a lambda to pass extra args
    ddf_embedded = ddf.map_partitions(
        lambda part: embed_partition(part, model, embedding_output_dimensions),
        meta={"text": str, "embedding": object}
    )
 
    # Trigger Dask to compute
    result_df = ddf_embedded.compute()
 
    return result_df["embedding"].tolist()

In [ ]:
data_dict = run_question_data_pipeline(data_map, raw_data, api_key,'wave_10',encoding_type='HuggingFace')


🚨 DEBUG: Inside process_question_guide
Shape of data_map: (786, 2)
Columns in data_map: ['Q80', 'Where do you live?']
First 5 rows:
   Q80 Where do you live?
0   1      United States
1   2     United Kingdom
2   3              Italy
3   4             France
4   5            Germany


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-ada-002 in organization org-Edw9Vvm6IxAT4DmEXFl12qii on tokens per min (TPM): Limit 10000000, Used 10000000, Requested 275. Please try again in 1ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}